# Deep Q Learning: Including Experience Replay and Target Network to the Neural Fitted Q Learning algorithm

## Set up display for OpenAI gym environments on Google Colab
More information on why this is done can be found in [this article](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7). 

In [ ]:
%%bash
# install required system dependencies
apt-get install -y xvfb x11-utils
pip install gym[box2d]==0.17.* \
            pyvirtualdisplay==0.2.* \
            PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.* > /dev/null 2>&1

In [ ]:
import pyvirtualdisplay

_display = pyvirtualdisplay.Display(visible=False,
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
# Check if a display port has indeed been set up. The output should be a number.
!echo $DISPLAY

## Build the RL algorithm

Initial imports and settings

In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

random.seed(10) # For repeatability

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("CarRacing-v0")

The default action space for this environment is continuous. Discretise it into a few different action options.

In [ ]:
# actions that agent can take
# manually chosen action from continuous action space to discrete action space
# 0: STEER LEFT
# 1: DO NOTHING
# 2: STEER RIGHT
# 3: FULL GAS
# 4: FULL BREAK
ACTIONSPACE = [[-1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
n_actions = len(ACTIONSPACE)

Pre-processing functions

In [ ]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])

# wait until env zooms to car
def wait_for_zoom():
    for x in range(0, 50):
        env.step([0, 0, 0])

def get_screen():
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    # Car is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.6):int(screen_height * 0.9)]
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, screen_width//3:2*screen_width//3]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0).to(DEVICE)

'''
# Testing the two above functions
env.reset()
wait_for_zoom()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()
'''

Define your neural network.

In [ ]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

Set the hyperparameters and some utilities

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

env.reset()
init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape

# Input: cropped image showing the state
# Output: Q value of each action (one action per output node)
policy_net = DQN(screen_height, screen_width, n_actions).to(DEVICE)


optimizer = optim.RMSprop(policy_net.parameters())


steps_done = 0

# Epsilon-greedy for action selection
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=DEVICE, dtype=torch.long)

In [ ]:
episode_returns = []
def plot_returns():
    plt.pause(0.01)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
    plt.figure(2)
    plt.clf()
    returns = torch.tensor(episode_returns, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Returns')
    plt.plot(returns.numpy())
    if len(returns) >= 10: #initially 100
        means = returns.unfold(0, 10, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(9), means))
        plt.plot(means.numpy())

Add Replay Memory to store previous experiences as transitions.

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Add the Target Network

In [ ]:
target_net = DQN(screen_height, screen_width, n_actions).to(DEVICE)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

memory = ReplayMemory(500)

Write your optimisation function.

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=DEVICE, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=DEVICE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute MSE loss
    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

## Train

In [ ]:
num_episodes = 20

for i_episode in range(num_episodes):
    wait_for_zoom()
    rewards_in_episode = 0
    print('Episode: '+str(i_episode))
    # Initialize the environment and state
    env.reset()
    last_screen = get_screen()
    # Make sure last_screen and current_screne are not equal, otherwise state will be just zeroes
    _, reward, done, _ = env.step([0,1,0])
    current_screne = get_screen()
    state = current_screne - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(ACTIONSPACE[action.item()])
        rewards_in_episode = rewards_in_episode + reward
        reward = torch.tensor([reward], device=DEVICE)

        # Observe new state
        last_screen = current_screne
        current_screne = get_screen()
        if not done:
            next_state = current_screne - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_returns.append(rewards_in_episode)
            print('Episode returns: ' + str(rewards_in_episode))
            plot_returns()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()

print('Saving model weights to ./policy_net')
torch.save(policy_net.state_dict(),'./policy_net')

plt.ioff()
plt.show()

## Inference

In [ ]:
loaded_policy_net = DQN(screen_height, screen_width, n_actions).to(DEVICE)
loaded_policy_net.load_state_dict(torch.load('./policy_net'))
loaded_policy_net.eval()

In [ ]:
from gym import wrappers

done = False
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
wait_for_zoom()
last_screen = get_screen()

# Push the car forward so that current screen and last screen are not equal
env.step([0, 1, 0])

current_screne = get_screen()
state = current_screne - last_screen

while not done:
    with torch.no_grad():

        _, reward, done, _ = env.step(ACTIONSPACE[loaded_policy_net(state).max(1)[1].view(1, 1).item()])

        reward = torch.tensor([reward], device=DEVICE)

        # Observe new state
        last_screen = current_screne
        current_screne = get_screen()
        if not done:
            next_state = current_screne - last_screen
        else:
            next_state = None

        # Move to the next state
        state = next_state
        if done: break

env.close()

Visualise the inference episode

In [ ]:
import io
import base64
from IPython.display import HTML

In [ ]:
video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))